In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2
%matplotlib inline

In [2]:
import os
import glob
import parse

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.nn as nn
import matplotlib.pyplot as plt

from functools import partial
from datetime import datetime
from modules.unet import UNet
from logger import VisdomLogger
from utils import *
from losses import soft_dice_loss
from metrics import F1_score

/home/kivva/miniconda/envs/py38/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [3]:
BUFFER_SIZE = 1000
BATCH_SIZE = 64
CROP_SIZE = 128
ROTATE = True
FLIP = True
MEAN_FILTER = False
AUTOTUNE = tf.data.experimental.AUTOTUNE
DOWNSAMPLE = 5
ONE_HOT = True

train_files = [(f"{DATA_DIR}{TRAIN_DIR}images/{file}", f"{DATA_DIR}{TRAIN_DIR}groundtruth/{file}") for file in os.listdir(f"{DATA_DIR}{TRAIN_DIR}images/") if "png" in file]
val_files = [(f"{DATA_DIR}{VAL_DIR}images/{file}", f"{DATA_DIR}{VAL_DIR}groundtruth/{file}") for file in os.listdir(f"{DATA_DIR}{VAL_DIR}images/") if "png" in file]

In [4]:
if ONE_HOT:
    mask_shape = [400, 400, 2]
else:
    mask_shape = [400, 400, 1]

dataset_generator = partial(dataset_generator_from_files, one_hot=ONE_HOT)

train_loader = tf.data.Dataset.from_generator(
    lambda: dataset_generator(train_files),
    output_types = (tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([400, 400, 3]), tf.TensorShape(mask_shape))
).shuffle(BUFFER_SIZE)

augment = partial(augmentation, crop_size=CROP_SIZE, rotate=ROTATE, flip=FLIP, mean_filter=MEAN_FILTER)
train_loader = train_loader.cache().batch(BATCH_SIZE)
train_loader = train_loader.map(augment, AUTOTUNE).prefetch(AUTOTUNE)

In [5]:
val_loader = tf.data.Dataset.from_generator(
    lambda: dataset_generator(val_files),
    output_types = (tf.float32, tf.float32),
    output_shapes=(tf.TensorShape([400, 400, 3]), tf.TensorShape(mask_shape))
).shuffle(BUFFER_SIZE)

val_loader = val_loader.cache().batch(BATCH_SIZE)
val_loader = val_loader.map(augment, AUTOTUNE).prefetch(AUTOTUNE)

In [6]:
EPOCHS = 3000
logdir = "logs/"

In [7]:
!rm -rf logs/
path_model = f"models/unet{DOWNSAMPLE}_binary_cross_entr"
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    loss = tf.keras.losses.BinaryCrossentropy()
    #loss = soft_dice_loss
    metrics = ["acc", F1_score]
    model = UNet(downsample=DOWNSAMPLE, out_channel=mask_shape[-1])
    model.compile(optimizer="Adam", loss=loss, metrics=metrics)
    
    learning_rate_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
    tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=logdir)
    
    model.fit(train_loader, epochs=EPOCHS, verbose=0, validation_data=val_loader, callbacks=[tensorboard_callbacks, learning_rate_callback])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for models/unet5_binary_cross_entr/unet6

In [ ]:
path_model = f"models/unet{DOWNSAMPLE}_binary_cross_entr"
model.save_weights(path_model+"/unet6")